# Operaton API

This notebook demonstrates how to interact with the Operaton (Camunda 7) REST API using `operaton.py`.

The module provides:
- `load_env()` - Load environment variables from localStorage
- `Operaton` - A class with static methods for REST API calls

## Process Definitions

List and inspect deployed process definitions.

In [ ]:
import operaton
from operaton import Operaton

# Load environment variables from localStorage
await operaton.load_env()

# Get all latest process definitions
definitions = Operaton.get('/process-definition?latestVersion=true')

print(f"Found {len(definitions)} process definition(s):")
for defn in definitions:
    print(f"  - {defn['key']} (v{defn['version']}): {defn.get('name', '(unnamed)')}")

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a dropdown for process selection
if definitions:
    process_dropdown = widgets.Dropdown(
        options=[(f"{d['key']} - {d.get('name', '(unnamed)')}", d['id']) for d in definitions],
        description='Process:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px')
    )
    display(process_dropdown)
else:
    print("No process definitions available.")

In [ ]:
# Show details of selected process
if definitions and 'process_dropdown' in dir():
    process_id = process_dropdown.value
    
    # Get process definition details
    details = Operaton.get(f'/process-definition/{process_id}')
    
    print("Process Definition Details:")
    print(f"  ID: {details['id']}")
    print(f"  Key: {details['key']}")
    print(f"  Name: {details.get('name', '(unnamed)')}")
    print(f"  Version: {details['version']}")
    print(f"  Deployment ID: {details['deploymentId']}")
    print(f"  Suspended: {details.get('suspended', False)}")

## View BPMN Diagram

Fetch and display the BPMN diagram for a process definition.

In [ ]:
import json

if definitions and 'process_dropdown' in dir():
    process_id = process_dropdown.value
    
    # Fetch BPMN XML
    xml_response = Operaton.get(f'/process-definition/{process_id}/xml')
    bpmn_xml = xml_response['bpmn20Xml']
    
    # Display configuration
    data = json.dumps({
        'style': {'height': '400px'},
        'zoom': 0.8
    })
    
    display({
        'application/bpmn+xml': bpmn_xml,
        'application/bpmn+json': data
    }, raw=True)
else:
    print("Select a process definition first.")

## Process Instances

List running process instances.

In [ ]:
# Get running process instances
instances = Operaton.get('/process-instance?maxResults=10')

if instances:
    print(f"Found {len(instances)} running process instance(s) (showing max 10):")
    for inst in instances:
        print(f"  - {inst['id']} ({inst['definitionId'][:30]}...)")
        print(f"    Business Key: {inst.get('businessKey', '(none)')}")
        print(f"    Suspended: {inst.get('suspended', False)}")
else:
    print("No running process instances.")

## User Tasks

List open user tasks.

In [ ]:
# Get open tasks
tasks = Operaton.get('/task?maxResults=10')

if tasks:
    print(f"Found {len(tasks)} open task(s) (showing max 10):")
    for task in tasks:
        print(f"  - {task['name']} ({task['id']})")
        print(f"    Assignee: {task.get('assignee', '(unassigned)')}")
        print(f"    Created: {task['created']}")
else:
    print("No open tasks.")

## Incidents

List active incidents.

In [ ]:
# Get incidents
incidents = Operaton.get('/incident?maxResults=10')

if incidents:
    print(f"Found {len(incidents)} incident(s) (showing max 10):")
    for incident in incidents:
        print(f"  - {incident['incidentType']} at {incident['activityId']}")
        print(f"    Message: {incident.get('incidentMessage', '(no message)')[:80]}")
        print(f"    Timestamp: {incident['incidentTimestamp']}")
else:
    print("No active incidents.")

## History

Query historical process instances.

In [ ]:
# Get completed process instances
history = Operaton.get('/history/process-instance?finished=true&maxResults=10')

if history:
    print(f"Found {len(history)} completed process instance(s) (showing max 10):")
    for hist in history:
        duration_ms = hist.get('durationInMillis', 0)
        duration_s = duration_ms / 1000 if duration_ms else 0
        print(f"  - {hist['processDefinitionKey']}")
        print(f"    Started: {hist['startTime']}")
        print(f"    Ended: {hist['endTime']}")
        print(f"    Duration: {duration_s:.1f}s")
else:
    print("No completed process instances in history.")

## Custom Query

Execute a custom REST API query.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import json

query_input = widgets.Text(
    value='/engine',
    description='Endpoint:',
    placeholder='/process-definition?latestVersion=true',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

query_button = widgets.Button(
    description='Execute',
    button_style='primary',
    icon='play'
)

query_output = widgets.Output()

def execute_query(button):
    with query_output:
        clear_output()
        try:
            result = Operaton.get(query_input.value)
            print(json.dumps(result, indent=2))
        except Exception as e:
            print(f"Error: {e}")

query_button.on_click(execute_query)

display(widgets.VBox([
    widgets.HBox([query_input, query_button]),
    query_output
]))